### 신경망 모델 구성하기
* 신경망: layer와 module로 구성 
* Pytorch의 모든 모듈은 nn.Module의 하위클래스  
  * 신경망은 다른 모듈(layer)로 구성된 모듈

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
#학습을 위한 장치
# torch.cuda를 사용할 수 있는지 확인하고 그렇지 않으면 cpu사용
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### 클래스 정의하기
* 신경망 모델을 nn.Module의 하위클래스로 정의하고,  ```__init__```
에서 신경망 계층들을 초기화  

* nn.Module를 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현함  

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512,10),
    )
    
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device)
print(X.shape)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

torch.Size([1, 28, 28])
Predicted class: tensor([5], device='cuda:0')


----

### 모델 계층(layer)


In [ ]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten  
* nn.Flatten계층을 초기화하여 각 28*28의 2D이미지를 784 픽셀 값을 갖는 연속된 배열로 변환함.

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [ ]:
class Flatten(torch.nn.Module):
  def forward(self, x):
    batch_size = x.shape[0]
    return x.view(batch_size, -1)

# f = Flatten() 

nn.Linear  
* 선형 계층은 저장된 가중치와 편향을 사용하여 입력에 선형변형(linear transformation)을 적용하는 모듈

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image) #nn.Linear에 위에서 flatten한 image를 넣어줌
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
* activation : model input과 output사이의 복잡한 관계(mapping)를 만듦.


In [ ]:
print(f"Before ReLU : {hidden1}\n")
hidden2 = nn.ReLU()(hidden1)
print(f"After ReLU : {hidden2}")

Before ReLU : tensor([[-0.0783,  0.4267, -0.1952,  0.0305,  0.3683,  0.1203, -0.3205,  0.2006,
         -0.2462, -0.0437, -0.2835, -0.1353, -0.1426,  0.5756,  0.0881, -0.0827,
          0.4027,  1.1206, -0.0272,  0.6430],
        [-0.2887,  0.1454, -0.3945,  0.1720,  0.3197, -0.1711, -0.4305,  0.3609,
         -0.0222, -0.0218, -0.0847, -0.3665,  0.0561,  0.0454,  0.4120, -0.1452,
          0.4667,  1.0118,  0.1075,  0.1443],
        [-0.2394,  0.5380, -0.3037, -0.2443,  0.4208,  0.0624, -0.1442,  0.5754,
         -0.1836, -0.0768, -0.3601, -0.1608, -0.0195,  0.4727,  0.2943,  0.0565,
          0.1974,  0.6396,  0.3220,  0.3145]], grad_fn=<AddmmBackward0>)

After ReLU : tensor([[0.0000, 0.4267, 0.0000, 0.0305, 0.3683, 0.1203, 0.0000, 0.2006, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.5756, 0.0881, 0.0000, 0.4027, 1.1206,
         0.0000, 0.6430],
        [0.0000, 0.1454, 0.0000, 0.1720, 0.3197, 0.0000, 0.0000, 0.3609, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0561, 0.0454, 0.4

nn.Sequential
* nn.Sequential은 순서를 갖는 모듈의 컨테이너  


In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1, 
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [ ]:
print(logits)

tensor([[-0.3423,  0.0803,  0.3334, -0.3385, -0.1712,  0.0085,  0.3474, -0.1136,
         -0.1698, -0.0052],
        [-0.2861,  0.0809,  0.2846, -0.3443, -0.3039,  0.0883,  0.3600, -0.2723,
         -0.2878,  0.1397],
        [-0.3228,  0.0555,  0.3610, -0.4039, -0.2651,  0.0473,  0.3621, -0.1605,
         -0.3114,  0.0192]], grad_fn=<AddmmBackward0>)


nn.Softmax 
* 신경망의 마지막 선형 계층은 nn.Softmax모듈에 전달될 logits를 반환함.


In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0717, 0.1095, 0.1410, 0.0720, 0.0851, 0.1019, 0.1430, 0.0902, 0.0852,
         0.1005],
        [0.0767, 0.1107, 0.1357, 0.0723, 0.0753, 0.1115, 0.1463, 0.0777, 0.0765,
         0.1174],
        [0.0744, 0.1085, 0.1473, 0.0686, 0.0788, 0.1077, 0.1475, 0.0875, 0.0752,
         0.1047]], grad_fn=<SoftmaxBackward0>)


모델 매개변수

In [ ]:
print(f"Model structure : {model}\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | size : {param.size()} | Values : {param[:2]}\n")

Model structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

Layer: linear_relu_stack.0.weight | size : torch.Size([512, 784]) | Values : tensor([[-0.0210, -0.0226,  0.0009,  ..., -0.0126,  0.0067,  0.0284],
        [ 0.0115, -0.0012,  0.0132,  ...,  0.0129,  0.0003, -0.0202]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | size : torch.Size([512]) | Values : tensor([-0.0078, -0.0187], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | size : torch.Size([512, 512]) | Values : tensor([[ 0.0405,  0.0124,  0.0432,  ...,  0.0397,  0.0048,  0.0182],
        [-0.0035,  0.0172,  0.0102,  ..., -0.0415,  0.0298, -0.0207]],
       device='cuda:0', grad_fn=<S

----

###* convolution block을 구현


In [ ]:
import torch 
import torch.nn as nn

###1. PyTorch 모델의 기본 구조 
* PyTorch 모델로 쓰기 위해선 다음 두 가지 조건을 따라야한다. 내장된 모델(nn.Linear등)도 이를 만족한다. 

1. torch.nn.Module을 상속해야한다. 
 * interitance: 상속; 어떤 클래스를 만들 때 다른 클래스의 기능을 그대로 가지고오는 것.  

2. __init()__과 forward()를 override 해야한다. 
 * override: 재정의; torch.nn.Module(부모클래스)에서 정의한 메소드를 자식클래스에서 변경하는 것. 
 * __init()__에서는 모델에서 사용될 module(nn.Linear, nn.Conv2d), activation function(nn.functional.relu, nn.functional.sigmoid)등을 정의한다. 
 * forward()에서는 모델에서 실행되어야하는 계산을 정의한다. backward 계산은 backward()를 이용하면 PyTorch가 알아서 해주니까 forward()만 정의해주면 된다. input을 넣어서 어떤 계산을 진행하하여 output이 나올지를 정의해준다고 이해하면 됨. 


### 2. nn.Module 
* PyTorch의 nn 라이브러리는 Neural Network의 모든 것을 포괄하는 모든 신경망 모델의 Base Class이다.  
다른 말로, 모든 신경망 모델은 nn.Module의 subclass라고 할 수 있다. 
nn.Module을 상속한 subclass가 신경망 모델로 사용되기 위해선 앞서 소개한 두 메소드를 override 해야한다. 

 * __init__(self): initialize; 내가 사용하고 싶은, 내 신경망 모델에 사용될 구성품들을 정의 및 초기화 하는 메소드이다. 
 * forward(self, x): specify the connections;  이닛에서 정의된 구성품들을 연결하는 메소드이다. 

In [ ]:
class ConvBlock(nn.Module):
'''
ConvBlock은 nn.Module을 상속받는다.

모듈이란, 한개 이상의 레이어가 모여서 구성된 것을 말함.

pytorch의 layer를 사용하여 모델 build를 간편하게 하기 위해서는 위에 같이 nn.Module를 상속받고, 
이를 초기화함으로써, nn.Module에서 상속받는 특성들을 초기화해주는 것이 필요하다.
'''

  def __init__(self, kernel_size, stride, padding, pool, pool_stride):
    super().__init__()   
    #super로 기반클래스(부모클래스)를 초기화 -> 기반클래스의 속성을 subclass가 받아오도록 함.
    #초기화 하지 않으면, 부모클래스의 속성을 사용할 수 없음

    '''__init__() 를 override(재정의)해야 함.  
    -> init에서는 모델에 사용될 module(nn.Linear, nn.Conv2d),activation function(nn.functional.relu) 등을 정의

    '''

    self.kernel_size = kernel_size
    self.padding = padding
    self.pool = pool
    self.pool_stride = pool_stride
    self.conv_block = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size = self.kernel_size, stride=self.stride,padding= self.padding),
        nn.BatchNormr2d(16),
        nn.ReLU(),
        nn.MaxPool2d(self.pool, self.pool_stride)
                  )
  
  def forward(self, x):
    '''
    forward(): 모델에서 실행되어야 하는 계산을 정의
    backward 계산은 backward()를 이용하면 pytorch가 자동으로 해주므로 forward()만 정의하면 됨.

    input을 넣어서 어떤 계산을 진행하여 output이 나올지 정의해줌.
    '''
    output= self.conv_block(x)
    return output
  
#model = ConvBlock(...)